In [10]:
import pandas as pd

In [11]:
def add_week_day(df, date_column="timestamp"):
    df = df.copy()
    df["week_day"] = df[date_column].dt.day_name()
    return df

In [78]:
def calcular_jornada(df, evento_inicio=1, evento_fin=4):
    jornada = df[df["evento_id"].isin([evento_inicio, evento_fin])].copy()
    jornada = jornada.sort_values("timestamp").reset_index(drop=True)

    # Cada evento de inicio inicia un bloque
    jornada["bloque"] = (jornada["evento_id"] == evento_inicio).cumsum()

    # Pivot para tener inicio y fin
    bloques = jornada.pivot_table(
        index=["bloque", jornada["timestamp"].dt.date, "dia_semana"],
        columns="evento_id",
        values="timestamp",
        aggfunc="first"
    )

    bloques = bloques.rename(columns={evento_inicio: "inicio", evento_fin: "fin"})

    # Solo bloques completos
    bloques = bloques.dropna(subset=["inicio", "fin"])

    # Duración del bloque
    bloques["duracion"] = bloques["fin"] - bloques["inicio"]

    # Reset index y renombrar columna de fecha
    bloques = bloques.reset_index()
    bloques = bloques.rename(columns={"timestamp": "fecha"})

    return bloques[["fecha", "dia_semana", "inicio", "fin", "duracion"]]

In [83]:
def calcular_concentracion(df, evento_inicio=2, evento_fin=3):
    trabajo = df[df["evento_id"].isin([evento_inicio, evento_fin])].copy()
    trabajo = trabajo.sort_values("timestamp").reset_index(drop=True)

    # Cada evento 2 inicia un bloque
    trabajo["bloque"] = (trabajo["evento_id"] == evento_inicio).cumsum()

    # Pivot para tener inicio y fin
    bloques = trabajo.pivot_table(
        index=["bloque", trabajo["timestamp"].dt.date, "dia_semana"],
        columns="evento_id",
        values="timestamp",
        aggfunc="first"
    )

    bloques = bloques.rename(columns={evento_inicio: "inicio", evento_fin: "fin"})

    # Solo bloques completos
    bloques = bloques.dropna(subset=["inicio", "fin"])

    # Duración del bloque
    bloques["duracion"] = bloques["fin"] - bloques["inicio"]

    bloques = bloques.reset_index()
    bloques = bloques.rename(columns={"timestamp": "fecha"})

    return bloques[["fecha", "dia_semana", "inicio", "fin", "duracion"]]

In [64]:
def resumen_diario(jornada_df, concentracion_df):
    jornada_sum = jornada_df.groupby("fecha")["duracion"].sum().rename("tiempo_jornada")
    concentracion_sum = concentracion_df.groupby("fecha")["duracion"].sum().rename("tiempo_concentracion")

    resumen = pd.concat([jornada_sum, concentracion_sum], axis=1).reset_index()

    # Opcional: horas en decimal
    resumen["horas_jornada"] = resumen["tiempo_jornada"].dt.total_seconds() / 3600
    resumen["horas_concentracion"] = resumen["tiempo_concentracion"].dt.total_seconds() / 3600

    return resumen

In [3]:
# Cargar CSV
df = pd.read_csv("concentration_tracking.csv")

In [8]:
df.head(10)

,date,time,event_id,timestamp,week_day
0,2026-01-09,9:36:29,1,2026-01-09 09:36:29,Friday
1,2026-01-09,9:36:33,2,2026-01-09 09:36:33,Friday
2,2026-01-09,9:43:53,3,2026-01-09 09:43:53,Friday
3,2026-01-09,10:00:57,2,2026-01-09 10:00:57,Friday
4,2026-01-09,10:39:52,3,2026-01-09 10:39:52,Friday
5,2026-01-09,11:00:25,2,2026-01-09 11:00:25,Friday
6,2026-01-09,11:40:43,3,2026-01-09 11:40:43,Friday
7,2026-01-09,11:51:58,2,2026-01-09 11:51:58,Friday
8,2026-01-09,12:18:47,3,2026-01-09 12:18:47,Friday
9,2026-01-09,12:34:10,4,2026-01-09 12:34:10,Friday


In [5]:
#consolidate date format
df["date"] = pd.to_datetime(df["date"].astype(str), format="mixed")

#combine date and time in a single column
df["timestamp"] = pd.to_datetime(
    df["date"].astype(str) + " " + df["time"].astype(str),
    format="mixed"
)

#order by timestamp
df = df.sort_values("timestamp").reset_index(drop=True)

In [9]:
df = add_week_day(df)

#dataframe overview
df.head(10)

,date,time,event_id,timestamp,week_day
0,2026-01-09,9:36:29,1,2026-01-09 09:36:29,Friday
1,2026-01-09,9:36:33,2,2026-01-09 09:36:33,Friday
2,2026-01-09,9:43:53,3,2026-01-09 09:43:53,Friday
3,2026-01-09,10:00:57,2,2026-01-09 10:00:57,Friday
4,2026-01-09,10:39:52,3,2026-01-09 10:39:52,Friday
5,2026-01-09,11:00:25,2,2026-01-09 11:00:25,Friday
6,2026-01-09,11:40:43,3,2026-01-09 11:40:43,Friday
7,2026-01-09,11:51:58,2,2026-01-09 11:51:58,Friday
8,2026-01-09,12:18:47,3,2026-01-09 12:18:47,Friday
9,2026-01-09,12:34:10,4,2026-01-09 12:34:10,Friday


In [80]:
df_j = calcular_jornada(df)
df_j

evento_id,fecha,dia_semana,inicio,fin,duracion
0,2026-01-09,Viernes,2026-01-09 09:36:29,2026-01-09 12:34:10,0 days 02:57:41
1,2026-01-09,Viernes,2026-01-09 15:04:43,2026-01-09 16:10:27,0 days 01:05:44
2,2026-01-12,Lunes,2026-01-12 09:38:29,2026-01-12 14:09:12,0 days 04:30:43
3,2026-01-12,Lunes,2026-01-12 15:02:23,2026-01-12 18:14:49,0 days 03:12:26


In [84]:
df_c = calcular_concentracion(df)
df_c

evento_id,fecha,dia_semana,inicio,fin,duracion
0,2026-01-09,Viernes,2026-01-09 09:36:33,2026-01-09 09:43:53,0 days 00:07:20
1,2026-01-09,Viernes,2026-01-09 10:00:57,2026-01-09 10:39:52,0 days 00:38:55
2,2026-01-09,Viernes,2026-01-09 11:00:25,2026-01-09 11:40:43,0 days 00:40:18
3,2026-01-09,Viernes,2026-01-09 11:51:58,2026-01-09 12:18:47,0 days 00:26:49
4,2026-01-09,Viernes,2026-01-09 15:04:45,2026-01-09 15:17:28,0 days 00:12:43
5,2026-01-09,Viernes,2026-01-09 15:20:40,2026-01-09 15:51:03,0 days 00:30:23
6,2026-01-09,Viernes,2026-01-09 16:01:03,2026-01-09 16:02:23,0 days 00:01:20
7,2026-01-12,Lunes,2026-01-12 09:38:31,2026-01-12 10:10:16,0 days 00:31:45
8,2026-01-12,Lunes,2026-01-12 10:29:21,2026-01-12 10:43:58,0 days 00:14:37
9,2026-01-12,Lunes,2026-01-12 10:49:44,2026-01-12 11:31:35,0 days 00:41:51


In [91]:
r_df = resumen_diario(df_j, df_c)
r_df["fecha"] = pd.to_datetime(r_df["fecha"].astype(str))

In [93]:
r_df = add_week_day(r_df, "fecha")
r_df

,fecha,tiempo_jornada,tiempo_concentracion,horas_jornada,horas_concentracion,dia_semana
0,2026-01-09,0 days 04:03:25,0 days 02:37:48,4.056944,2.630000,Viernes
1,2026-01-12,0 days 07:43:09,0 days 05:12:42,7.719167,5.211667,Lunes
